In [1]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

def normalize(vectors):
    """Normalize vectors using L2 normalization"""
    return vectors / np.linalg.norm(vectors, axis=1, keepdims=True)

def create_and_save_job_embeddings(csv_path, model_name="sentence-transformers/all-MiniLM-L6-v2",
                                   faiss_path="job_index.faiss", df_path="job_df.pkl"):
    """
    Create and save job embeddings from a CSV file
    
    Args:
        csv_path (str): Path to CSV with 'Job Title' and 'Job Description'
        model_name (str): SentenceTransformer model
        faiss_path (str): Output path for FAISS index
        df_path (str): Output path to save job_df (pickled DataFrame)
    """
    # Load model
    model = SentenceTransformer(model_name)

    # Load CSV
    job_df = pd.read_csv(csv_path)
    job_df['job_text'] = job_df['Job Title'] + " " + job_df['Job Description']

    # Encode & Normalize
    job_texts = job_df['job_text'].tolist()
    print("🔄 Creating embeddings...")
    embeddings = model.encode(job_texts, show_progress_bar=True)
    embeddings = normalize(embeddings)

    # Create FAISS index
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(embeddings.astype(np.float32))

    # Save
    faiss.write_index(index, faiss_path)
    job_df.to_pickle(df_path)
    print(f"✅ Saved FAISS index to '{faiss_path}' and job_df to '{df_path}'")

# Example usage:
create_and_save_job_embeddings("job_title_des.csv")


c:\Users\ASUS VivoBook\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🔄 Creating embeddings...


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

✅ Saved FAISS index to 'job_index.faiss' and job_df to 'job_df.pkl'
